In [4]:
%pip install -r requirements.txt

^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ------------- -------------------------- 3.9/11.5 MB 21.3 MB/s eta 0:00:01
   ------------------------------ --------- 8.9/11.5 MB 22.2 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 21.4 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 21.4 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 21.4 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 21.4 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 21.4 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 21.4 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 6.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ------------- -------------------------- 4.2/12.6 MB 21.0 MB/s eta 0:00:01
   

# Classes

In [ ]:
from datetime import datetime
from dateutil.relativedelta import (
    relativedelta,
)  # Função que calcula direto a idade em anos, meses e dias.
import pandas as pd
import os

from datetime import datetime
from dateutil.relativedelta import relativedelta

class Paciente:
    """
    Classe que modela a entidade paciente dentro de um sistema de gestão hospitalar.
    """

    def __init__(
        self,
        id_paciente: str,
        nome: str,
        cpf: str,
        data_nascimento: str,
        sexo: str,
        peso: float,
        altura: float,
    ) -> None:
        self.id = id_paciente
        self.nome = nome
        self.cpf = cpf
        self.data_nascimento = data_nascimento
        self.idade = self.calcular_idade(data_nascimento)
        self.sexo = sexo
        self.peso = peso
        self.altura = altura
        self.imc = self.calcular_imc(peso, altura)
        self.consultas = []

    def calcular_imc(self, peso: float, altura: float) -> float:
        if altura <= 0:  
            raise ValueError("Altura deve ser maior que zero para calcular o IMC.")
        return round(peso / (altura**2), 2)

    def calcular_idade(self, data_nascimento: str) -> str:
        nascimento = datetime.strptime(data_nascimento, "%d/%m/%Y")
        hoje = datetime.now()
        idade = relativedelta(hoje, nascimento)
        return f"{idade.years} anos, {idade.months} meses e {idade.days} dias"

    def __call__(self):
        return {
            "id": self.id,
            "nome": self.nome,
            "cpf": self.cpf,
            "data_nascimento": self.data_nascimento,
            "idade": self.idade,
            "sexo": self.sexo,
            "peso": self.peso,
            "altura": self.altura,
            "imc": self.imc,
            "consultas": self.consultas,
        }

class Funcionario:
    """
    Classe que modela a entidade funcionário dentro de um sistema de gestão hospitalar
    """

    def __init__(self, id_Funcionario: str, nome: str, cargo: str, n_doc: int) -> None:
        self.id = id_Funcionario
        self.nome = nome
        self.cargo = cargo
        self.n_doc = n_doc


class Radiofarmaco:
    """
    Classe que modela a entidade radiofármaco dentro de um sistema de gestão hospitalar
    """

    def __init__(
        self,
        id_radiofarmaco: str,
        princp_ativo: str,
        concentracao: str,
        data_fabricacao: str,
    ) -> None:
        self.id = id_radiofarmaco
        self.princip_ativo = princp_ativo
        self.concentracao = concentracao
        self.data_fabricacao = data_fabricacao


class Exame:
    """
    Classe que modela a entidade exame dentro de um sistema de gestão hospitalar
    """

    def __init__(
        self,
        id_exame: str,
        tipo: str,
        data: str,
        id_paciente: str,
        id_funcionario: str,
        id_consulta: str,
        id_radiofarmaco: str,
    ) -> None:
        self.id = id_exame
        self.tipo = tipo
        self.data = data
        self.id_paciente = id_paciente
        self.id_funcionario = id_funcionario
        self.id_consulta = id_consulta
        self.id_radiofarmaco = id_radiofarmaco


class Passagem:
    def __init__(
        self,
        id_consulta: str,
        id_paciente: str,
        data: str,
        id_funcionario: str,
        procedimento: list[str],
    ) -> None:
        self.id_consulta = id_consulta
        self.id_paciente = id_paciente
        self.data = data
        self.id_funcionario = id_funcionario
        self.procedimento = procedimento


class BancoDados:
    """
    Classe que modela o banco de dados utilizando dataframes e salvando em .csv
    """

    def __init__(self, nome: str) -> None:
        self.nome = nome
        self.df: pd.DataFrame = pd.DataFrame()
        if not os.path.exists("data"):
            os.makedirs("data")
        self.caminho_arquivo = "data/" + self.nome + ".csv"

    def adicionar_coluna(self, nome_coluna: str, valor_coluna: str):
        """
        Adiciona uma coluna ao dataframe, contanto que a coluna não exista.
        Caso exista, retorna False
        """
        if nome_coluna in self.df.columns:
            print("Erro: A coluna já existe.")
            return False
        else:
            self.df[nome_coluna] = valor_coluna
        return self.df

    def adicionar_linha(self, linha: list[dict]) -> pd.DataFrame:
        """
        Adiciona uma linha ao dataframe, contanto que a linha esteja no formato do dataframe.
        Caso não esteja, retorna False
        """
        df_linha = pd.DataFrame(linha)
        if self.df.empty:
            self.df = df_linha
        elif (self.df.columns == df_linha.columns).all():
            self.df = pd.concat([self.df,df_linha], ignore_index=True)
        else:
            print("Erro: A linha não corresponde ao formato do dataframe.")
            return False

    def salvar(self):
        """
        Salva o dataframe em um arquico .csv.
        """

        self.df.to_csv(self.caminho_arquivo, index=False)

    def atualizar_banco(self) -> pd.DataFrame:
        """
        Carrega um arquivo .csv em um dataframe.

        """

        self.df = pd.read_csv(self.caminho_arquivo)

        return self.df

    def carregar_dados(
        self, caminho_dados: str = None, dados: dict = None
    ) -> pd.DataFrame:
        """
        Carrega dados de um arquivo .csv ou de um dicionário em um dataframe.

        """
        if caminho_dados is not None:
            self.df = pd.read_csv(caminho_dados)
        elif dados is not None:
            self.df = pd.DataFrame(dados)
        else:
            print("Erro: Nenhum dado fornecido para carregar.")

        return self.df

    def __call__(self):
        return self.df

# Cadastrar pacientes


In [34]:
paciente1 = Paciente('001',"João Gomes","123.456.789-10", "18/12/1950", "M", 80.5, 1.822)
paciente2 = Paciente('002', "Joaquina Gomes", "123.456.789-11", "18/12/1970", "F", 54, 1.6)
paciente3 = Paciente('003', "Bruna Alves", "123.456.789-50", "28/11/2004", "F", 74, 1.52)

paciente_db = BancoDados('pacientes')
paciente_db.adicionar_linha([paciente1(), paciente2()])
paciente_db.adicionar_linha([paciente3()])

paciente_db()

,id,nome,cpf,data_nascimento,idade,sexo,peso,altura,imc,consultas
0,001,João Gomes,123.456.789-10,18/12/1950,"73 anos, 11 meses e 10 dias",M,80.5,1.822,24.25,[]
1,002,Joaquina Gomes,123.456.789-11,18/12/1970,"53 anos, 11 meses e 10 dias",F,54.0,1.600,21.09,[]
2,003,Bruna Alves,123.456.789-50,28/11/2004,"20 anos, 0 meses e 0 dias",F,74.0,1.520,32.03,[]
